# 模块C：协议处理与转换（Protocol）测试

覆盖：HTTP/1.1、HTTP/2、gRPC(h2c)、WebSocket、协议转换、改写/镜像/聚合、压缩转换、缓存、批处理/拆分/流式响应。

提示：默认使用 `../build` 目录；如需重新编译，把 `BUILD_FIRST = True`。

In [ ]:
from __future__ import annotations

import os
import subprocess
from pathlib import Path

ROOT = (Path("..") if Path.cwd().name == "notebooks" else Path(".")).resolve()
BUILD_DIR = (ROOT / "build").resolve()

TIMEOUT_S = 30
BUILD_FIRST = False
CONTINUE_ON_FAIL = False

TESTS = [
    # basic HTTP server & parsing
    "test_http",
    "test_httpserver",
    "test_http_keepalive_chunked",
    # HTTP/2 / gRPC / WS
    "test_http2_basic",
    "test_grpc_h2c",
    "test_websocket_upgrade",
    # conversion / content processing
    "test_protocol_conversion",
    "test_compression_conversion",
    "test_rewrite_rules",
    "test_traffic_mirror",
    "test_api_aggregate",
    # caching
    "test_cache_memcached",
    "test_cache_redis",
    # batching / split / streaming
    "test_http_batching",
    "test_batch_split",
    "test_streaming_response",
    # routing features
    "test_cookie",
    "test_model_version_routing",
]


In [ ]:
def sh(cmd: list[str], *, cwd: Path = ROOT, timeout: int | None = None) -> None:
    print("$", " ".join(cmd))
    subprocess.run(cmd, cwd=str(cwd), check=True, timeout=timeout)


def maybe_build() -> None:
    BUILD_DIR.mkdir(parents=True, exist_ok=True)
    cache = BUILD_DIR / "CMakeCache.txt"
    if not cache.exists():
        sh(["cmake", "-S", str(ROOT), "-B", str(BUILD_DIR)])
    jobs = str(max(1, os.cpu_count() or 4))
    sh(["cmake", "--build", str(BUILD_DIR), "-j", jobs])


def run_tests() -> None:
    failures: list[tuple[str, str]] = []
    for t in TESTS:
        p = BUILD_DIR / t
        if not p.exists():
            print(f"SKIP {t} (missing: {p})")
            continue
        print(f"RUN {t}")
        try:
            sh([str(p)], cwd=ROOT, timeout=TIMEOUT_S)
        except subprocess.TimeoutExpired:
            print(f"FAIL {t} timeout={TIMEOUT_S}s")
            failures.append((t, "timeout"))
            if not CONTINUE_ON_FAIL:
                break
        except subprocess.CalledProcessError as e:
            print(f"FAIL {t} exit={e.returncode}")
            failures.append((t, f"exit={e.returncode}"))
            if not CONTINUE_ON_FAIL:
                break

    if failures:
        raise RuntimeError("Failed tests: " + ", ".join([f"{n}({r})" for n, r in failures]))
    print("OK")


if BUILD_FIRST:
    maybe_build()
run_tests()
